# Final Project: End-to-End Data Cleaning Workflow

### Farmers Market Dataset

> Author: *Luis Magana* (luism2)

- Repository: https://github.com/maganaluis/CS513-FinalProject
- Overview & Assesment: https://github.com/maganaluis/CS513-FinalProject/blob/master/InitialAssesment.pdf

In [1]:
from IPython.display import display
import pandas as pd
import numpy as np
import re
import calendar
import datetime

In [2]:
df = pd.read_csv('farmers.csv')

In [3]:
df.shape #(Rows, Cols)

(8739, 59)

### Raw Data View

In [4]:
with pd.option_context('display.max_columns', None):
    display(df.head(4))

,FMID,MarketName,Website,Facebook,Twitter,Youtube,OtherMedia,street,city,County,State,zip,Season1Date,Season1Time,Season2Date,Season2Time,Season3Date,Season3Time,Season4Date,Season4Time,x,y,Location,Credit,WIC,WICcash,SFMNP,SNAP,Organic,Bakedgoods,Cheese,Crafts,Flowers,Eggs,Seafood,Herbs,Vegetables,Honey,Jams,Maple,Meat,Nursery,Nuts,Plants,Poultry,Prepared,Soap,Trees,Wine,Coffee,Beans,Fruits,Grains,Juices,Mushrooms,PetFood,Tofu,WildHarvested,updateTime
0,1018261,Caledonia Farmers Market Association - Danville,https://sites.google.com/site/caledoniafarmers...,https://www.facebook.com/Danville.VT.Farmers.M...,NaN,NaN,NaN,NaN,Danville,Caledonia,Vermont,05828,06/14/2017 to 08/30/2017,Wed: 9:00 AM-1:00 PM;,09/06/2017 to 10/18/2017,Wed: 2:00 PM-6:00 PM;,NaN,NaN,NaN,NaN,-72.140337,44.411036,NaN,Y,Y,N,Y,N,Y,Y,Y,Y,Y,Y,N,Y,Y,Y,Y,Y,Y,N,N,N,Y,Y,Y,Y,N,Y,Y,Y,N,N,Y,Y,N,N,6/20/2017 10:43:57 PM
1,1018318,Stearns Homestead Farmers' Market,http://www.StearnsHomestead.com,StearnsHomesteadFarmersMarket,NaN,NaN,NaN,6975 Ridge Road,Parma,Cuyahoga,Ohio,NaN,06/24/2017 to 09/30/2017,Sat: 9:00 AM-1:00 PM;,NaN,NaN,NaN,NaN,NaN,NaN,-81.733939,41.374801,NaN,Y,N,N,Y,N,-,Y,N,Y,Y,Y,N,Y,Y,Y,Y,Y,N,N,N,N,Y,N,Y,N,N,N,N,Y,N,N,N,N,N,N,6/21/2017 5:15:01 PM
2,1009364,106 S. Main Street Farmers Market,http://thetownofsixmile.wordpress.com/,NaN,NaN,NaN,NaN,106 S. Main Street,Six Mile,NaN,South Carolina,29682,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-82.818700,34.804200,NaN,Y,N,N,N,N,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013
3,1010691,10th Steet Community Farmers Market,NaN,NaN,NaN,NaN,http://agrimissouri.com/mo-grown/grodetail.php...,10th Street and Poplar,Lamar,Barton,Missouri,64759,04/02/2014 to 11/30/2014,Wed: 3:00 PM-6:00 PM;Sat: 8:00 AM-1:00 PM;,NaN,NaN,NaN,NaN,NaN,NaN,-94.274619,37.495628,NaN,Y,N,N,N,N,-,Y,N,Y,N,Y,N,Y,Y,Y,Y,N,Y,N,N,Y,Y,Y,Y,N,N,N,N,Y,N,N,N,N,N,N,10/28/2014 9:49:46 AM


## Data Cleaning


All of my data cleaning can be done with Pandas, since the dataset is small enough and is just one table. In order to clean the data and ensure it is on a usable format for the end user I've performed the following operations:

- Replace all nan values with '' (empty strings), as it is easier to handle strings with Python.
- Ensure updateTime is of DATETIME type and then format it to YYYY-MM-DD 
- Create FullAddress from columns street, city, zip and State only if they all are present, otherwise
  set the row to empty.
- Ensure that only Y or N are used in all bin columns, which are the products that are offered in the market. 
  Any value that is not Y defaults to N.
- Using regex ensure that the columns Website, Facebook and Youtube follow a URL format.
- Ensure that Twitter uses @username format.
- Ensure that season date uses a MM/DD/YYYY to MM/DD/YYYY format and that its season time is present.

In [5]:
#replace all nan values with empty strings for now
df = df.replace(np.nan, '', regex=True)

In [6]:
#ensure dates are properly formatted
df['updateTime'] = pd.to_datetime(df['updateTime']) #does the heavy lifting
df['updateTime'] = df['updateTime'].apply(lambda x: x.strftime("%Y-%m-%d"))

In [7]:
def form_address(rows):
    """
    Form address will return the address using the columns
    listed below, if any of the columns is not available
    it will return an empty string. 
    """
    cols = ['street', 'city', 'State', 'zip']
    if any([not rows[c] for c in cols]):
        return ''
    else:
        return ', '.join([rows[c] for c in cols])

#create standard address
address = df.apply(form_address, axis=1)
idx = df.columns.get_loc('zip') + 1
df.insert(idx, 'FullAddress', address)

In [8]:
#ensure bin columns are Y or N, if they are not Y defaults to N
bin_cols = df.iloc[:,23:-1].columns
for col in bin_cols:
    mask = mask = np.argwhere(df[col] != 'Y').ravel().tolist()
    df.loc[mask, col] = "N"

In [9]:
def is_valid_url(url):
    """
    Takes any string and returns it if it's a valid url
    returns and empty string otherwise.
    """
    regex = re.compile(
        r'^https?://'  # http:// or https://
        r'(?:(?:[A-Z0-9](?:[A-Z0-9-]{0,61}[A-Z0-9])?\.)+[A-Z]{2,6}\.?|'  # domain...
        r'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3})' # ...or ip
        r'(?::\d+)?'  # optional port
        r'(?:/?|[/?]\S+)$', re.IGNORECASE)
    if regex.search(url):
        return url
    else:
        return ''

In [10]:
#validate website, facebook, and youtube
df['Facebook'] = df['Facebook'].apply(is_valid_url)
df['Website'] = df['Website'].apply(is_valid_url)
df['Youtube'] = df['Youtube'].apply(is_valid_url)

In [11]:
#ensure twitter uses @
def format_twitter(row):
    if 'https://twitter.com/' in row:
        return row.replace('https://twitter.com/', '@')
    elif row and '@' not in row:
        return '@' + row
    elif '@' in row:
        return row
    else:
        return ''

df['Twitter'] = df['Twitter'].apply(format_twitter)
#othermedia can stay as is, since it's a free form input

In [12]:
#drop x, y geo locations since a lot appear to be incorrect
df = df.drop(columns=['x','y'])

In [13]:
def is_valid_season(row):
    """
    Returns the row if the season in the format 
    "%m/%d/%Y" to "%m/%d/%Y", otherwise it sends the 
    row to another function for further inspecting.
    """
    pattern = re.compile("\d+/\d+/\d+ to \d+/\d+/\d+")
    if pattern.match(row.strip()):
        return row
    else:
        return get_valid_season(row)

In [14]:
def get_valid_season(row):
    """
    This is the second step for season dates inspection, 
    if the season is formatted with names of months it will use
    the current year for the season, otherwise it will send an 
    empty string back.
    """
    vals = row.strip().split(" to ")
    months = {v: k for k,v in enumerate(calendar.month_name)}
    del months[''] #empty not a valid month
    if not vals:
        return ''
    elif not all([val in list(months.keys()) for val in vals]):
        return ''
    elif len(vals) > 2: #we are only expecting two
        return ''
    else:
        year = datetime.datetime.now().year
        dt1 = datetime.datetime(year=year, month=months[vals[0]], day=1)
        dt2 = datetime.datetime(year=year, month=months[vals[1]], day=1)
        return dt1.strftime("%m/%d/%Y") + " to " + dt2.strftime("%m/%d/%Y")

In [15]:
#validate and correct all seasons dates
df['Season1Date'] = df['Season1Date'].apply(is_valid_season)
df['Season2Date'] = df['Season2Date'].apply(is_valid_season)
df['Season3Date'] = df['Season3Date'].apply(is_valid_season)
df['Season4Date'] = df['Season4Date'].apply(is_valid_season)

## YesWorkFlow Model Graph

Below you can see the overall structure of the code, and you can find the source code for this model in the github repository as **WorkFlow.py**

The key input of this model is *farmers.csv*, which is fed into read_csv a powerful pandas connector. From there output is saved as a DataFrame, which is the basic data structure of pandas. The DataFrame will be fed into different methods that take a row, and return an output in functional programming fashion.

Each method is independent and will write back to the same DataFrame with exception of `Drop_Geo_Locators` which doesn't provide an actual output. In the end the same DataFrame which will be used to create a new *CleanFarmers.csv*

<img style="height:500px;" src="https://raw.githubusercontent.com/maganaluis/CS513-FinalProject/master/FarmersWf.png">

### Clean Data View

In [16]:
with pd.option_context('display.max_columns', None):
    display(df.head(4))

,FMID,MarketName,Website,Facebook,Twitter,Youtube,OtherMedia,street,city,County,State,zip,FullAddress,Season1Date,Season1Time,Season2Date,Season2Time,Season3Date,Season3Time,Season4Date,Season4Time,Location,Credit,WIC,WICcash,SFMNP,SNAP,Organic,Bakedgoods,Cheese,Crafts,Flowers,Eggs,Seafood,Herbs,Vegetables,Honey,Jams,Maple,Meat,Nursery,Nuts,Plants,Poultry,Prepared,Soap,Trees,Wine,Coffee,Beans,Fruits,Grains,Juices,Mushrooms,PetFood,Tofu,WildHarvested,updateTime
0,1018261,Caledonia Farmers Market Association - Danville,https://sites.google.com/site/caledoniafarmers...,https://www.facebook.com/Danville.VT.Farmers.M...,,,,,Danville,Caledonia,Vermont,05828,,06/14/2017 to 08/30/2017,Wed: 9:00 AM-1:00 PM;,09/06/2017 to 10/18/2017,Wed: 2:00 PM-6:00 PM;,,,,,N,Y,Y,N,Y,N,Y,Y,Y,Y,Y,Y,N,Y,Y,Y,Y,Y,Y,N,N,N,Y,Y,Y,Y,N,Y,Y,Y,N,N,Y,Y,N,N,2017-06-20
1,1018318,Stearns Homestead Farmers' Market,http://www.StearnsHomestead.com,,,,,6975 Ridge Road,Parma,Cuyahoga,Ohio,,,06/24/2017 to 09/30/2017,Sat: 9:00 AM-1:00 PM;,,,,,,,N,Y,N,N,Y,N,N,Y,N,Y,Y,Y,N,Y,Y,Y,Y,Y,N,N,N,N,Y,N,Y,N,N,N,N,Y,N,N,N,N,N,N,2017-06-21
2,1009364,106 S. Main Street Farmers Market,http://thetownofsixmile.wordpress.com/,,,,,106 S. Main Street,Six Mile,,South Carolina,29682,"106 S. Main Street, Six Mile, South Carolina, ...",,,,,,,,,N,Y,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,2013-01-01
3,1010691,10th Steet Community Farmers Market,,,,,http://agrimissouri.com/mo-grown/grodetail.php...,10th Street and Poplar,Lamar,Barton,Missouri,64759,"10th Street and Poplar, Lamar , Missouri, 64759",04/02/2014 to 11/30/2014,Wed: 3:00 PM-6:00 PM;Sat: 8:00 AM-1:00 PM;,,,,,,,N,Y,N,N,N,N,N,Y,N,Y,N,Y,N,Y,Y,Y,Y,N,Y,N,N,Y,Y,Y,Y,N,N,N,N,Y,N,N,N,N,N,N,2014-10-28


## Logical Integrity Constraints (ICs)

With Pandas is easy to create a database schema by using "create_engine" and save the farmers.db file for later use. In this case I've identified the following integrity constrains that I think we should enforce:

- FMID must be unique
- All rows must have a MarketName
- All markets must have at least one season 
- If a market has one season they must provide a time
- All markets must provide city and state
- If a market doesn't provide an address it must provide at least a form of media

> Note: We are using the __clean__ DataFrame for this section.

In [17]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///farmers.db') #Notice is an SQLite Engine

In [18]:
df.replace('', np.nan, inplace=True) #set back empty strings to NAN, it is easier to handle these on SQL
df.to_sql('farmers', con=engine, if_exists='replace', index=False)

First we need to make sure that FMID entries are unique, altough I've verified this on Pandas we can also check with SQL and as we can see all of them are.

In [19]:
engine.execute("""SELECT CASE WHEN COUNT(DISTINCT FMID)= COUNT(FMID)
                         THEN 'column values are unique' ELSE 'column values are NOT unique' END
                         FROM farmers""").fetchall()

[('column values are unique',)]

We need to ensure that all markets have a name, it would be rare to see one without it and as we can see they all do. 

In [20]:
engine.execute("""SELECT COUNT(*) FROM farmers WHERE MarketName IS NULL""").fetchall()

[(0,)]

Next we need to make sure that every market has at least one season, this lets the user know when the market is available or open without having to check their website. We can see that 3299 of them don't have any season. 

Because it is a large number we should not drop these items, but ensure that at least one season is entered in future entries.

In [21]:
engine.execute("""SELECT COUNT(*) FROM farmers AS least_one_season WHERE
                          ((CASE WHEN Season1Date IS NULL THEN 0 ELSE 1 END) +
                          (CASE WHEN Season2Date IS NULL THEN 0 ELSE 1 END) +
                          (CASE WHEN Season3Date IS NULL THEN 0 ELSE 1 END) +
                          (CASE WHEN Season4Date IS NULL THEN 0 ELSE 1 END)) = 0
                           """).fetchall()

[(3299,)]

Now from the entries that have at least one season we must find the the ones that do not have at least one season time. As we can see there are 81 entries, for which we can obtain the FMID and then decide if we need to drop them.

In [22]:
engine.execute("""SELECT COUNT(*) FROM
                    ((SELECT * FROM farmers WHERE
                          ((CASE WHEN Season1Date IS NULL THEN 0 ELSE 1 END) +
                           (CASE WHEN Season2Date IS NULL THEN 0 ELSE 1 END) +
                           (CASE WHEN Season3Date IS NULL THEN 0 ELSE 1 END) +
                           (CASE WHEN Season4Date IS NULL THEN 0 ELSE 1 END)) > 0) AS LSONE)
                    WHERE 
                          ((CASE WHEN LSONE.Season1Time IS NULL THEN 0 ELSE 1 END) +
                           (CASE WHEN LSONE.Season2Time IS NULL THEN 0 ELSE 1 END) +
                           (CASE WHEN LSONE.Season3Time IS NULL THEN 0 ELSE 1 END) +
                           (CASE WHEN LSONE.Season3Time IS NULL THEN 0 ELSE 1 END)) = 0
                           """).fetchall()

[(81,)]

Since we need to know where the market is located, we need to ensure they provide at very least a city and a state. In this case we've found 40 entries that don't meet this criteria. We can also obtain the FMID and drop these entries.

In [23]:
engine.execute("""SELECT COUNT(*) FROM farmers
                    WHERE city IS NULL OR State IS NULL""").fetchall()

[(40,)]

Finally some markets can have multiple locations (streets) per season, therefore we can allow entries not to contain a street. If this is the case we assume that they are providing the end-user the location in their website or social media. In this case we found 117 entries that do not have a street and do not provide any social media.

In [24]:
engine.execute("""SELECT COUNT(*) FROM 
                  ((SELECT * FROM farmers WHERE
                  ((CASE WHEN Website IS NULL THEN 0 ELSE 1 END) +
                   (CASE WHEN Facebook IS NULL THEN 0 ELSE 1 END) +
                   (CASE WHEN Twitter IS NULL THEN 0 ELSE 1 END) +
                   (CASE WHEN Youtube IS NULL THEN 0 ELSE 1 END) +
                   (CASE WHEN OtherMedia IS NULL THEN 0 ELSE 1 END)) = 0) AS LSONE)
                   WHERE LSONE.street IS NULL""").fetchall()

[(117,)]